# Taller Python para aprendizaje máquina y big data

### 8 Escuela Nacional de Aprendizaje e Inteligencia Computacional
### 28 de septiembre de 2021

In [1]:
!wget personal.cimat.mx:8181/~alejandro.rosales/resource/adult.csv

--2021-09-29 01:38:52--  http://personal.cimat.mx:8181/~alejandro.rosales/resource/adult.csv
Resolving personal.cimat.mx (personal.cimat.mx)... 189.203.35.250
Connecting to personal.cimat.mx (personal.cimat.mx)|189.203.35.250|:8181... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5339297 (5.1M) [text/csv]
Saving to: ‘adult.csv’

adult.csv           100%[===================>]   5.09M  1.39MB/s    in 4.6s    

2021-09-29 01:38:57 (1.12 MB/s) - ‘adult.csv’ saved [5339297/5339297]



In [ ]:
# Importando la librería pandas y asignandoles el alias de 'pd'
import pandas as pd

In [ ]:
# Cargando los datos de entrenamiento 
dataset = pd.read_csv('adult.csv')

In [ ]:
# Determinar el número de ejemplos de entrenamiento y características en la base de datos de Iris
num_samples, num_features = dataset.shape
print('El número de ejemplos de entrenamiento es %d, los cuales están descritos por %d características' %(num_samples, num_features))

El número de ejemplos de entrenamiento es 48842, los cuales están descritos por 15 características


In [ ]:
# Observando los datos de entrenamiento
print('Los datos de entrenamiento del conjunto adult son:')
print(dataset)

# Observando el valor medio de cada característica
print('El valor medio de cada atributo está dado por:')
print(dataset.mean(axis = 0))

# Observando la varianza de cada atributo
print('La varianza de cada atributo es:')
print(dataset.std(axis = 0))

# Obteniendo descriptores de los datos
dataset_stats = dataset.describe()
print(dataset_stats)

Los datos de entrenamiento del conjunto adult son:
       Age     Workclass  Fnlwgt  ... Hours-per-week  Native-country Class 
0       25       Private  226802  ...             40   United-States  <=50K
1       38       Private   89814  ...             50   United-States  <=50K
2       28     Local-gov  336951  ...             40   United-States   >50K
3       44       Private  160323  ...             40   United-States   >50K
4       18          NaN   103497  ...             30   United-States  <=50K
...    ...           ...     ...  ...            ...             ...    ...
48837   27       Private  257302  ...             38   United-States  <=50K
48838   40       Private  154374  ...             40   United-States   >50K
48839   58       Private  151910  ...             40   United-States  <=50K
48840   22       Private  201490  ...             20   United-States  <=50K
48841   52  Self-emp-inc  287927  ...             40   United-States   >50K

[48842 rows x 15 columns]
El valor m

In [ ]:
# Graficando los boxplot de las características
dataset.boxplot()

In [ ]:
# Obteniendo el nombre de las características
feature_names = dataset.columns
print(feature_names)

In [ ]:
# Accesando a la información de una columna en particular
dataset['Age']

In [ ]:
# Obteniendo las clases únicas del conjunto de datos
class_labels = dataset['Class '].unique()
print(class_labels)

In [ ]:
print(feature_names)

In [ ]:
dataset.isna().any()

In [ ]:
print(dataset)

In [ ]:
# Importamos la librería numpy con el alias np
import numpy as np

# Reemplazamos los valores faltantes no reconocidos por NaN
dataset.replace([r"\s*\s", r'\\'], [np.nan, np.nan], regex = True, inplace = True)
dataset

In [ ]:
dataset.isna()

In [ ]:
""" Alternativas para manejo de datos faltantes
1. Remover los ejemplos faltantes
2. Reemplazar los valores faltantes por un valor fijo
3. Reemplazar los valores faltantes por el de su vecino"""

# Eliminar los ejemplos con valores faltantes
dataset_dropped = dataset.copy()
dataset_dropped.dropna(axis = 0, how = 'any', inplace = True)
dataset_dropped


In [ ]:
dataset.isna().any()

In [ ]:
# Reemplazar con un valor fijo
dataset_mean_fill = dataset.copy()
dataset_mean_fill.fillna({'Workclass': 'Private', 'Occupation': 'Craft-repair', 'Native-country': 'Mexico'}, inplace = True)
dataset_mean_fill

In [ ]:
dataset

In [ ]:
# Reemplazar por el valor de un vecino
dataset_fill_neig = dataset.copy()
dataset_fill_neig.fillna(method = 'ffill', inplace = True)
dataset_fill_neig

In [ ]:
# Verificamos que alguno de los conjuntos editados ya no existan valores faltantes
dataset_fill_neig.isna().any()

In [ ]:
# Determinando el tipo de variables que se tiene
dataset_fill_neig.columns

In [ ]:
dataset_fill_neig.dtypes

In [ ]:
dataset_fill_neig['Workclass'].unique()

In [ ]:
# Los atributos categóricos son convertidos a numérico

# importamos el método OneHotEncoder de la librería de sklearn
from sklearn.preprocessing import LabelEncoder

LabelEncoder().fit_transform(dataset_fill_neig['Class '])


dataset_filled_num = dataset_fill_neig.copy()
for feature in dataset_fill_neig.columns:
    if dataset_fill_neig[feature].dtypes.name == 'object':
        dataset_filled_num[feature] = LabelEncoder().fit_transform(dataset_fill_neig[feature])
#        dataset_filled_num[feature] = dataset_filled_num[feature].astype('int64')

dataset_filled_num

In [ ]:
# Entrenando un clasificador
from sklearn.tree import DecisionTreeClassifier

X = dataset_filled_num[dataset_filled_num.columns[:-1]]
y = dataset_filled_num[dataset_filled_num.columns[-1]]
clf = DecisionTreeClassifier(max_depth = 3).fit(X, y)

y_hat = clf.predict(X)

In [ ]:
# Evaluando el modelo de clasificación

# Importando las métricas de evaluación
from sklearn.metrics import accuracy_score, f1_score
acc = accuracy_score(y, y_hat)
f1 = f1_score(y, y_hat, average = 'macro')
print('%s alcanza un %1.4f en exactitud y %1.4f en F1' % (type(clf), acc, f1))

In [ ]:
# Definiendo particiones de entrenamiento y prueba
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)

[y_train.shape, y_test.shape]

In [ ]:
# Normalizando los datos: todas las características en un rango común
from sklearn.preprocessing import MinMaxScaler # Todas las características media cero y desviación unitaria

scaler = MinMaxScaler().fit(X_train)
X_train_std, X_test_std = scaler.transform(X_train), scaler.transform(X_test)

In [ ]:
X_train_std

In [ ]:
# Entrenando y evaluando los clasificadores DT, KNN, RF y evaluando con exactitud y F1

#Importando los clasificadores
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier


classifiers = [DecisionTreeClassifier(), KNeighborsClassifier(n_neighbors = 3), RandomForestClassifier(n_estimators = 50)]
for clf in classifiers:
    clf.fit(X_train_std, y_train)
    y_hat = clf.predict(X_test_std)
    acc = accuracy_score(y_test, y_hat)
    f1 = f1_score(y_test, y_hat, average = 'macro')
    print('%s alcanza un %1.4f en exactitud y %1.4f en F1' % (type(clf), acc, f1))

In [ ]:
# Usando validación cruzada
from sklearn.model_selection import StratifiedKFold

kf = StratifiedKFold(n_splits = 2)

X = np.array(dataset_filled_num[dataset_filled_num.columns[:-1]])
y = np.array(dataset_filled_num[dataset_filled_num.columns[-1]])

fold = 1
for train_index, test_index in kf.split(X, y):
    X_train, y_train = X[train_index], y[train_index]
    X_test, y_test = X[test_index], y[test_index]
    scaler = MinMaxScaler().fit(X_train)
    X_train_std, X_test_std = scaler.transform(X_train), scaler.transform(X_test)
    classifiers = [DecisionTreeClassifier(), KNeighborsClassifier(n_neighbors = 3), RandomForestClassifier(n_estimators = 50)]
    for clf in classifiers:
        clf.fit(X_train_std, y_train)
        y_hat = clf.predict(X_test_std)
        acc = accuracy_score(y_test, y_hat)
        f1 = f1_score(y_test, y_hat, average = 'macro')
        print('%s alcanza un %1.4f en exactitud y %1.4f en F1 en el fold %d' % (type(clf), acc, f1, fold))
    fold += 1
    